In [26]:
# File: interactive_paint_with_dynamic_brush.py
import cv2
import mediapipe as mp
from mediapipe.tasks.python.vision.gesture_recognizer import GestureRecognizerResult
import numpy as np
import math
import time

# Inicialización de Mediapipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

model_path = './gesture_recognizer.task'
base_options = mp.tasks.BaseOptions(model_asset_path=model_path)

# Configuración de colores
colors = [(0, 0, 255), (0, 255, 0), (255, 0, 0), (0, 255, 255)]  # Rojo, Verde, Azul, Amarillo
current_color = colors[0]
brush_size = 10

# Variables del menú y pintura
menu_width = 100
drawing = False
canvas = None  # Para almacenar lo dibujado
dominant_hand = None  # Será "Left" o "Right"
calibrating = True  # Modo calibración

I0000 00:00:1736157136.433726  203972 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1736157136.449707  225208 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.0.9-0ubuntu0.2), renderer: Mesa Intel(R) UHD Graphics 620 (WHL GT2)


In [27]:
# Funciones auxiliares
def draw_menu(frame):
    # Dibuja los colores del menú sin fondo.
    for i, color in enumerate(colors):
        cx, cy = 50, 100 + i * 100
        cv2.circle(frame, (cx, cy), 30, color, -1)

def count_visible_fingers(hand_landmarks):
    # Cuenta el número de dedos visibles.
    finger_tips = [
        mp_hands.HandLandmark.THUMB_TIP,
        mp_hands.HandLandmark.INDEX_FINGER_TIP,
        mp_hands.HandLandmark.MIDDLE_FINGER_TIP,
        mp_hands.HandLandmark.RING_FINGER_TIP,
        mp_hands.HandLandmark.PINKY_TIP
    ]

    visible_fingers = 0
    for tip in finger_tips:
        tip_coord = hand_landmarks.landmark[tip]
        base_coord = hand_landmarks.landmark[tip - 2]  # Base de cada dedo
        if tip_coord.y < base_coord.y:  # Dedo levantado si la punta está por encima de la base
            visible_fingers += 1

    return visible_fingers

def calculate_brush_size(hand_landmarks):
    # Calcula el tamaño de la brocha según la distancia entre los dedos índice y medio.
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]

    # Distancia euclidiana entre los dos dedos
    distance = math.sqrt((index_tip.x - middle_tip.x)**2 + (index_tip.y - middle_tip.y)**2)
    return int(distance * 300)  # Escalar la distancia para que sea visible

def gesture_callback(result: GestureRecognizerResult, output_image: mp.Image, timestamp_ms: int):
    # print(result)

    if not result.gestures:
        return
    
    global calibrating
    if calibrating:
        dominant_hand = detect_dominant_hand(result)

        if dominant_hand:
            calibrating = False
            print('dominant hand set to ', dominant_hand)


def detect_dominant_hand(result):
    # Detecta si la mano es izquierda o derecha.
    if result.hand_landmarks and result.handedness and result.gestures:
        # Check each hand
        for hand_idx in range(len(result.hand_landmarks)):
            hand_label = result.handedness[hand_idx][0].category_name
            gesture_name = result.gestures[hand_idx][0].category_name
            
            if gesture_name == "Open_Palm":
                return hand_label
            
    # If no hand with Open_Palm was found
    return None

In [28]:
options = mp.tasks.vision.GestureRecognizerOptions(
    base_options=base_options,
    running_mode=mp.tasks.vision.RunningMode.LIVE_STREAM,
    result_callback=gesture_callback
)
recognizer = mp.tasks.vision.GestureRecognizer.create_from_options(options)

# Iniciar cámara
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if canvas is None:
        canvas = np.zeros_like(frame)  # Inicializar el lienzo

    current_timestamp_ms = int(time.time() * 1000)
    frame = cv2.flip(frame, 1)  # Flip horizontal para parecer espejo
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)
    frame_height, frame_width, _ = frame.shape

    if calibrating:
        # Mostrar mensaje para calibrar
        cv2.putText(frame, "Muestra tu mano dominante", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)
        recognizer.recognize_async(mp_image, current_timestamp_ms)

    else:
        # Detección de manos (solo rastrear la dominante)
        if results.multi_handedness and results.multi_hand_landmarks:
            for i, hand_landmarks in enumerate(results.multi_hand_landmarks):
                handedness = results.multi_handedness[i].classification[0].label
                if handedness != dominant_hand:
                    continue  # Ignorar manos que no sean la dominante

                # Coordenadas del índice
                index_finger = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                x, y = int(index_finger.x * frame_width), int(index_finger.y * frame_height)

                # Contar dedos visibles
                visible_fingers = count_visible_fingers(hand_landmarks)

                if visible_fingers == 0:
                    # Puño cerrado, desactivar dibujo
                    drawing = False
                else:
                    # Ajustar el tamaño del pincel dinámicamente
                    drawing = True
                    if visible_fingers == 1:
                        brush_size = 10  # Tamaño mínimo para un dedo
                    elif visible_fingers >= 2:
                        brush_size = calculate_brush_size(hand_landmarks)

                # Detectar interacción con el menú
                wrist_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x * frame_width)
                if wrist_x < menu_width:
                    # Cambiar color si se toca un círculo de color
                    for i, color in enumerate(colors):
                        cx, cy = 50, 100 + i * 100
                        if math.hypot(x - cx, y - cy) < 30:
                            current_color = color
                else:
                    # Dibujar según la posición del índice
                    if drawing:
                        cv2.circle(canvas, (x, y), brush_size, current_color, -1)  # Pintar

                # Mostrar el color del índice
                cv2.circle(frame, (x, y), 10, current_color, -1)  # Colorear la punta del índice

    # Combinar el lienzo con el frame
    frame = cv2.addWeighted(frame, 0.5, canvas, 0.5, 0)

    # Dibujar el menú
    draw_menu(frame)

    # Mostrar el frame
    cv2.imshow("Paint Interactivo", frame)

    key = cv2.waitKey(1)
    if key & 0xFF == 27:  # Presionar ESC para salir
        break
    elif key & 0xFF == ord('d'):  # Activar/desactivar dibujo
        drawing = not drawing

cap.release()
cv2.destroyAllWindows()


W0000 00:00:1736157136.643866  225201 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1736157136.664525  203972 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1736157136.668087  225211 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.0.9-0ubuntu0.2), renderer: Mesa Intel(R) UHD Graphics 620 (WHL GT2)
W0000 00:00:1736157136.670109  203972 gesture_recognizer_graph.cc:129] Hand Gesture Recognizer contains CPU only ops. Sets HandGestureRecognizerGraph acceleration to Xnnpack.
I0000 00:00:1736157136.676753  203972 hand_gesture_recognizer_graph.cc:250] Custom gesture classifier is not defined.
W0000 00:00:1736157136.707322  225201 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1736157136.736032  225213 inference_feedback_manager.cc:11

dominant hand set to  Left
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
Paint
P